In [1]:
from pydub import AudioSegment

# Load the mp4 file
audio = AudioSegment.from_file("TCS Campus Interview l Campus Placements l Gauri Shrimali.mp4")

# Save the audio as a wave file
audio.export("converted_audio.wav", format="wav")

<_io.BufferedRandom name='converted_audio.wav'>

In [2]:
"TCS Campus Interview l Campus Placements l Gauri Shrimali.mp4"[-4:]

'.mp4'

In [3]:
# converts mp4 to wav

def convert(mp4_path="", wav_save_path=""):
    assert mp4_path[-3:].lower() == "mp4"

    # load mp4
    audio = AudioSegment.from_file(mp4_path)
    
    # export to dir
    audio.export(wav_save_path, format="wav")

    return True

In [4]:
import wave
import os

# List of tuples representing start times and end times
times = [(202, 219), (232,234), (246, 264),(275, 283), (289, 301), (309, 313), (317, 334), (346, 357), (374, 404), (411, 420), (426, 439), (447, 450), (460, 467), (495, 496), (575, 596),(770, 830), (850, 877)]

# Open the wave file
with wave.open("converted_audio.wav", "rb") as wave_file:
    # Get the number of frames and the frame rate
    n_frames = wave_file.getnframes()
    frame_rate = wave_file.getframerate()
    # print(frame_rate)
    # # frame_rate = wave_file.getframerate()
    print(n_frames, frame_rate)
    
    # Create the temp folder if it doesn't exist
    if not os.path.exists("temp_clips"):
        os.mkdir("temp_clips")
    
    # Iterate over the start and end times
    for i, (start, end) in enumerate(times):
        # Calculate the start and end frame indices
        start_frame = int(start * frame_rate)
        end_frame = int(end * frame_rate)
        
        # print(start, end, start_frame, end_frame)
        
        # Skip the iteration if the start frame is greater than the end frame
        if start_frame >= end_frame:
            continue
        
        # print(wave_file.getnframes())
        # print(start_frame * frame_rate)
        
        # Read the audio data for the specified frame range
        wave_file.setpos(start_frame)
        audio_data = wave_file.readframes(end_frame - start_frame)
        
        # Save the audio data to a new wave file
        with wave.open(f"temp_clips/part{i}.wav", "wb") as output_file:
            output_file.setnchannels(wave_file.getnchannels())
            output_file.setsampwidth(wave_file.getsampwidth())
            output_file.setframerate(frame_rate)
            output_file.writeframes(audio_data)


48006144 44100


In [5]:
# make clips/batches(based on timestamps) of audio for easier processing
from typing import List, Tuple

def divide_clips(timestamps:List[Tuple], wav_path: str, temp_file="temp_clips"):
    # Open the wave file
    with wave.open(wav_path, "rb") as wave_file:
        # Get the number of frames and the frame rate
        n_frames = wave_file.getnframes()
        frame_rate = wave_file.getframerate()
        # print(frame_rate)
        # # frame_rate = wave_file.getframerate()
        print(n_frames, frame_rate)
        
        # Create the temp folder if it doesn't exist
        if not os.path.exists(f"{temp_file}"):
            os.mkdir(f"{temp_file}")
        
        # Iterate over the start and end times
        for i, (start, end) in enumerate(times):
            # Calculate the start and end frame indices
            start_frame = int(start * frame_rate)
            end_frame = int(end * frame_rate)
            
            # print(start, end, start_frame, end_frame)
            
            # Skip the iteration if the start frame is greater than the end frame
            if start_frame >= end_frame:
                continue
            
            # print(wave_file.getnframes())
            # print(start_frame * frame_rate)
            
            # Read the audio data for the specified frame range
            wave_file.setpos(start_frame)
            audio_data = wave_file.readframes(end_frame - start_frame)
            
            # Save the audio data to a new wave file
            with wave.open(f"{temp_file}/part{i}.wav", "wb") as output_file:
                output_file.setnchannels(wave_file.getnchannels())
                output_file.setsampwidth(wave_file.getsampwidth())
                output_file.setframerate(frame_rate)
                output_file.writeframes(audio_data)


In [6]:
from speechbrain.pretrained.interfaces import foreign_class
classifier = foreign_class(source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP", pymodule_file="custom_interface.py", classname="CustomEncoderWav2vec2Classifier")

d:\,Msc\AU Hackathon\ing_2023_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
torchvision is not available - cannot save figures
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
d:\,Msc\AU Hackathon\ing_2023_env\lib\site-packages\transformers\configuration_utils.py:375: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2

In [7]:
out_prob, score, index, text_lab = classifier.classify_file("temp_clips/part5.wav")

def get_res(key):
    if key.lower() == "hap":
        return "happy"
    elif key.lower() == "neu":
        return "neutral"
    elif key.lower() == "ang":
        return 'sadness'

print(score.numpy()[0]*100, text_lab[0], get_res(text_lab[0]))

100.0 neu neutral


In [8]:
import os
import time

clips_dir = "./temp_clips/"
clips = [os.path.join(clips_dir, clip_name) for clip_name in os.listdir(clips_dir)]

for idx, clip in enumerate(clips):
    ti = time.time()

    out_prob, score, index, text_lab = classifier.classify_file(clip)
    print(f"completed part{idx} in {time.time()-ti}", out_prob, score, index, text_lab)

completed part0 in 5.036293983459473 tensor([[3.8591e-10, 6.5445e-11, 1.0000e+00, 5.6274e-11]]) tensor([1.]) tensor([2]) ['hap']
completed part1 in 0.45429539680480957 tensor([[1.0000e+00, 4.4267e-09, 2.1922e-11, 4.5764e-12]]) tensor([1.]) tensor([0]) ['neu']
completed part2 in 3.5776400566101074 tensor([[1.4944e-03, 9.4728e-01, 5.1221e-02, 2.0022e-08]]) tensor([0.9473]) tensor([1]) ['ang']
completed part3 in 0.6679790019989014 tensor([[1.0000e+00, 3.7589e-08, 3.6911e-11, 3.6263e-12]]) tensor([1.]) tensor([0]) ['neu']
completed part4 in 1.5679395198822021 tensor([[5.8935e-02, 1.7947e-01, 7.6159e-01, 8.8963e-08]]) tensor([0.7616]) tensor([2]) ['hap']
completed part5 in 0.269132137298584 tensor([[1.4070e-02, 9.8593e-01, 1.8089e-09, 9.5158e-12]]) tensor([0.9859]) tensor([1]) ['ang']
completed part6 in 6.299887180328369 tensor([[5.6483e-06, 2.0207e-09, 9.9999e-01, 2.9471e-09]]) tensor([1.0000]) tensor([2]) ['hap']
completed part7 in 24.440092086791992 tensor([[5.3013e-09, 3.6017e-10, 1.000

In [9]:
# load model and make predictions over all the clips

def make_inferences():
    # loading model
    classifier = foreign_class(source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP", pymodule_file="custom_interface.py", classname="CustomEncoderWav2vec2Classifier")

    # make inferences over all the clips
    clips_dir = "./temp_clips/"
    clips = [os.path.join(clips_dir, clip_name) for clip_name in os.listdir(clips_dir)]

    for idx, clip in enumerate(clips):
        ti = time.time()

        out_prob, score, index, text_lab = classifier.classify_file(clip)
        print(f"completed part{idx} in {time.time()-ti}", score.numpy()[0], text_lab[0])

In [10]:
import gc;gc.collect()

105

In [11]:
# Speaker Recognition

import torchaudio
from speechbrain.pretrained import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")

In [12]:
signal, fs = torchaudio.load("temp_clips/part5.wav")
embeddings = classifier.encode_batch(signal)


In [13]:
from speechbrain.pretrained import SpeakerRecognition
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="pretrained_models/spkrec-ecapa-voxceleb")
score, prediction = verification.verify_files("temp_clips/part5.wav", "temp_clips/part4.wav")
print("Part4, 5", score, prediction)

score, prediction = verification.verify_files("temp_clips/part5.wav", "temp_clips/part6.wav")
print("Part5, 6", score, prediction)


Part4, 5 tensor([0.7764]) tensor([True])
Part5, 6 tensor([0.7837]) tensor([True])


In [14]:
import numpy as np
(np.math.comb(10, 2) * 15) / 60

11.25

Citation

In [15]:
# speech to text

In [16]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
model.config.forced_decoder_ids = None

In [17]:
# # load dummy dataset and read audio files
# # ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
# # sample = ds[0]["audio"]
# input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features 

# # generate token ids
# predicted_ids = model.generate(input_features)
# # decode token ids to text
# transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)
# print(transcription)

# transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
# print(transcription)


In [18]:
import librosa

def speech2text(wav_path, sample_rate=16000, skip_special_tokens=True, load_model=False):
    global model, processor
    if load_model:
        # load model
        processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
        model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
        model.config.forced_decoder_ids = None

    waveform, _ = librosa.load(wav_path, sr=sample_rate)
    input_features = processor(waveform, sampling_rate=sample_rate, return_tensors="pt").input_features 

    # generate token ids
    predicted_ids = model.generate(input_features)
    # decode token ids to text
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=skip_special_tokens)
    # print(transcription)
    return transcription


In [19]:
import os
import time

clips_dir = "./temp_clips/"
clips = [os.path.join(clips_dir, clip_name) for clip_name in os.listdir(clips_dir)]

ti = time.time()
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
model.config.forced_decoder_ids = None
print(f"time taken to load model: {time.time()-ti}s.".title())

for idx, clip in enumerate(clips):
    ti = time.time()

    # print(f"completed part{idx} in {time.time()-ti}", out_prob, score, index, text_lab)
    print(f"completed part{idx} in {time.time()-ti}\n".title(), speech2text(clip, skip_special_tokens=False))

Time Taken To Load Model: 85.91420125961304S.


d:\,Msc\AU Hackathon\ing_2023_env\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Completed Part0 In 0.0
 ["<|startoftranscript|><|en|><|transcribe|><|notimestamps|> said there is a work life balance and it promotes like gender equality. There is no like discrimination on the basis of that and so it's very good in ethics overall.<|endoftext|>"]
Completed Part1 In 0.0
 ['<|startoftranscript|><|en|><|transcribe|><|notimestamps|> Captain Hounso.<|endoftext|>']
Completed Part2 In 0.0
 ['<|startoftranscript|><|en|><|transcribe|><|notimestamps|> Yes, changes its technologies according to the needs of the people of the art. So there is no fixed technology that they use, they use according to their clients, which is<|endoftext|>']
Completed Part3 In 0.0
 ["<|startoftranscript|><|en|><|transcribe|><|notimestamps|> So, but I'm not aware of the clients.<|endoftext|>"]
Completed Part4 In 0.0
 ['<|startoftranscript|><|en|><|transcribe|><|notimestamps|> So, will I be able to like fit in the company? I mean, what do you think?<|endoftext|>']
Completed Part5 In 0.0
 ['<|startoftran

In [22]:
speech2text("converted_audio.wav", skip_special_tokens=False)

['<|startoftranscript|><|en|><|transcribe|> Yes, go ring. Tell me, tell me something old you. Thank you for allowing me to introduce myself. My name is Gauri Shumadi. I am from Odevo Rajasthan. I am doing computer science engineering from Gitanjali Institute of Technical Studies. I secured 9.2 CGPA in 10 standard 73.4 CG percentage and 12 standard and currently in engineering my advocates score is 77 percentage. Computer science is omnipresent.<|endoftext|>']

In [24]:
import torch
from transformers import pipeline

# from datasets import load_dataset

device = "cuda:0" if torch.cuda.is_available() els  e "cpu"

pipe = pipeline(
  "automatic-speech-recognition",
  model="openai/whisper-tiny",
  chunk_length_s=30,
  device=device,
)


In [25]:
# sample requires path, array and sampling rate
sample = {}
sample['path'] = "./converted_audio.wav"
sample['array'], sample["sampling_rate"] = librosa.load("./converted_audio.wav", sr=16000)

In [26]:
prediction = pipe(sample, return_timestamps=True)["chunks"]

d:\,Msc\AU Hackathon\ing_2023_env\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [27]:
prediction

[{'text': ' Yes, go ring. Tell me, tell me something old you.',
  'timestamp': (0.0, 3.0)},
 {'text': ' Okay, thank you for allowing me to introduce myself. My name is Gauri Shumadi.',
  'timestamp': (3.0, 8.0)},
 {'text': " I am from Odevo Rajasthan. I'm doing computer science engineering from Gitanjali Institute of Technical Studies.",
  'timestamp': (8.0, 15.0)},
 {'text': ' I secured 9.2 CGPA intense standard 73.4 cg percentage in 12 standard and currently in engineering my aggregates code is 77 percentage.',
  'timestamp': (15.0, 27.52)},
 {'text': ' Computer science is omnipresent that is it is present in every field and hence I have invested',
  'timestamp': (27.52, 33.12)},
 {'text': ' these last 3.5 almost years in developing my software skills. My technical skills include the programming in Python, Java, C, C++,',
  'timestamp': (33.12, 45.0)},
 {'text': ' databases.', 'timestamp': (45.0, 46.2)},
 {'text': ' I am also familiar with web development,',
  'timestamp': (46.2, 49.

In [28]:
import pandas as pd

In [29]:
df = pd.DataFrame(prediction)

In [30]:
df

,text,timestamp
0,"Yes, go ring. Tell me, tell me something old ...","(0.0, 3.0)"
1,"Okay, thank you for allowing me to introduce ...","(3.0, 8.0)"
2,I am from Odevo Rajasthan. I'm doing computer...,"(8.0, 15.0)"
3,I secured 9.2 CGPA intense standard 73.4 cg p...,"(15.0, 27.52)"
4,Computer science is omnipresent that is it is...,"(27.52, 33.12)"
...,...,...
241,So thank you.,"(1086.56, 1087.56)"
242,"All right, all set, right?","(1087.56, 1088.56)"
243,"Okay, sir.","(1088.56, 1089.56)"
244,Thank you.,"(1089.56, 1090.56)"
